<a href="https://colab.research.google.com/github/luanakwon/ML_Pytorch_pre2022/blob/main/TorchPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST 
---
Predicting hand written numbers with CNN model.  
  
  
(pytorch prctice with mnist data. didn't even know how to use matplotlib nor cv2 back then)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 

In [ ]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

In [ ]:
mnist_train = dset.MNIST("./",train=True, transform=transforms.ToTensor(),target_transform=None, download=True)
mnist_test = dset.MNIST("./",train=False, transform=transforms.ToTensor(),target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
z_len = 50

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.layer1 = nn.Linear(z_len, 4*4*64)
    self.layer2 = nn.Sequential(
        nn.Conv2d(64,32,)
    )

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3,100),
        nn.ReLU(),
        nn.Linear(100,10)
    )
  def forward(self, x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
loss_array = []

for i in range(num_epoch):
  for j, [image,label] in enumerate(train_loader):
    x = image.to(device)
    y_= label.to(device)
    print(x.shape)
    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j%1000==0:
      print(loss)
      loss_array.append(loss.cpu().detach().numpy())



torch.Size([256, 1, 28, 28])
tensor(2.3063, grad_fn=<NllLossBackward>)
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])


KeyboardInterrupt: ignored

In [ ]:
save_path = './drive/MyDrive/omok/modelsave.pth'
torch.save(model.state_dict(),save_path)

In [ ]:
save_path = './drive/MyDrive/omok/modelsave.pth'
new_model = CNN().to(device)
new_model.load_state_dict(torch.load(save_path,map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
    x = image.to(device)
    y_ = label.to(device)

    output = new_model(x)
    _,output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == y_).sum().float()
    print(label.shape)

  print("Accuracy of Test Data: {}".format(correct/total))  

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
Accuracy of Test Data: 0.9881811141967773


In [ ]:
import numpy as np

def print_image(image):
  im = np.array(image)
  color = [' ', '.', ':', '|', '@', '@']

  im *= (len(color) - 1)

  for r in im:
    for e in r:
      print(color[int(e)],end=' ')
    print()



In [ ]:
with torch.no_grad():
  for j, [image,label] in enumerate(test_loader):
    x = image.to(device)
    y_= label.to(device)

    output = new_model(x)
    _,output_index = torch.max(output,1)
    print_image(image[0,0])
    print("Answer is {0}, predict:{1}".format(label[0],output_index[0]))



                                                        
                                                        
                                                        
                                                        
                                                        
                                                        
                                                        
            . | | : .                                   
            @ @ @ @ @ @ | | | | | | | | | .             
            . : . : | @ @ @ @ @ @ @ @ @ @ :             
                        .   . . . .   @ @ :             
                                    . @ @               
                                    @ @ .               
                                  : @ @                 
                                . @ @ .                 
                                : @ |                   
                                @ @ .                   
                              :